## implementação dos da funcionalidade de zoom e os botões de zoom 

In [52]:
import tkinter as tk
from tkinter import filedialog, messagebox
import xml.etree.ElementTree as ET

class Visualizador:
    def __init__(self, root):
        self.root = root
        self.root.title("Visualizador de Objetos 2D")
    
        # Configurar o menu
        menu = tk.Menu(root)
        root.config(menu=menu)
        file_menu = tk.Menu(menu, tearoff=False)
        menu.add_cascade(label="Arquivo", menu=file_menu)
        file_menu.add_command(label="Abrir", command=self.abrir_arquivo)
        file_menu.add_command(label="Gerar XML de Saída", command=self.gerar_xml_saida)
    
        # Frame principal para conter canvas e minimapa
        frame_principal = tk.Frame(root)
        frame_principal.pack(side="top", fill="both", expand=True)
    
        # canvas da Viewport principal
        self.canvas = tk.Canvas(frame_principal, width=800, height=600, bg="white")
        self.canvas.pack(side="left", fill="both", expand=True)
    
        # canvas do minimapa na lateral direita
        self.minimap = tk.Canvas(frame_principal, width=150, height=120, bg="lightgrey") # razão 4:3
        self.minimap.pack(side="right", padx=10, pady=10)

        # frame para os botões de navegação na parte inferior
        frame_botoes = tk.Frame(root)
        frame_botoes.pack(side="bottom", anchor="w", padx=10, pady=10)

        # botões de navegação
        btn_up = tk.Button(frame_botoes, text="↑", width=3, command=lambda: self._mover_e_recarregar(0, 1))
        btn_down = tk.Button(frame_botoes, text="↓", width=3, command=lambda: self._mover_e_recarregar(0, -1))
        btn_left = tk.Button(frame_botoes, text="←", width=3, command=lambda: self._mover_e_recarregar(-1, 0))
        btn_right = tk.Button(frame_botoes, text="→", width=3, command=lambda: self._mover_e_recarregar(1, 0))

        # posiciona os botões em uma grade
        btn_up.grid(row=0, column=1, padx=3, pady=3)
        btn_left.grid(row=1, column=0, padx=3, pady=3)
        btn_right.grid(row=1, column=2, padx=3, pady=3)
        btn_down.grid(row=2, column=1, padx=3, pady=3)

        # botões de Zoom
        btn_zoom_in = tk.Button(frame_botoes, text="+", width=3, command=lambda: self._zoom(0.9)) 
        btn_zoom_out = tk.Button(frame_botoes, text="-", width=3, command=lambda: self._zoom(1.1))
        btn_zoom_in.grid(row=1, column=3, padx=(15, 3))
        btn_zoom_out.grid(row=1, column=4, padx=3)

        # lista de objetos lidos do XML
        self.objects = []       
        self.window = {'wmin':(0.0,0.0), 'wmax':(1.0,1.0)}
        self.viewport = {'vpmin':(0.0,0.0), 'vpmax':(800.0,600.0)}
        # matriz 3x3 world->viewport
        self.M = [[1.0,0.0,0.0],[0.0,1.0,0.0],[0.0,0.0,1.0]]
        # limites do mundo baseados nos objetos
        self.world_bounds = {'xmin': 0.0, 'xmax': 1.0, 'ymin': 0.0, 'ymax': 1.0}
        # caminho do arquivo carregado
        self.caminho_arquivo = None

        # Bind das teclas de seta para mover a window
        root.bind("<Left>", lambda e: self._mover_e_recarregar(-1,0))
        root.bind("<Right>", lambda e: self._mover_e_recarregar(1,0))
        root.bind("<Up>", lambda e: self._mover_e_recarregar(0,1))
        root.bind("<Down>", lambda e: self._mover_e_recarregar(0,-1))

        # Bind das teclas de zoom
        root.bind("<plus>", lambda e: self._zoom(0.9))
        root.bind("<equal>", lambda e: self._zoom(0.9))
        root.bind("<KP_Add>", lambda e: self._zoom(0.9))

        root.bind("<minus>", lambda e: self._zoom(1.1))
        root.bind("<KP_Subtract>", lambda e: self._zoom(1.1))

    # apenas abrir o arquivo e chamar a função para carregar dados...
    def abrir_arquivo(self):
        caminho = filedialog.askopenfilename(filetypes=[("XML files","*.xml"),("All files","*.*")])
        if not caminho:
            return
        try:
            self.caminho_arquivo = caminho  # armazena o caminho do arquivo
            self.carregar_arquivo(caminho)
        except Exception as e:
            messagebox.showerror("Erro", f"Erro ao carregar arquivo:\n{e}")

    # ler os dados do arquivo aberto
    def carregar_arquivo(self, caminho):
        tree = ET.parse(caminho)
        root = tree.getroot()

        # parse viewport
        vp = root.find('viewport')
        if vp is not None:
            vpmin = vp.find('vpmin')
            vpmax = vp.find('vpmax')
            if vpmin is not None and vpmax is not None:
                try:
                    self.viewport['vpmin'] = (float(vpmin.get('x', '0')), float(vpmin.get('y', '0')))
                    self.viewport['vpmax'] = (float(vpmax.get('x', '800')), float(vpmax.get('y', '600')))
                except ValueError:
                    # mantem valores anteriores se houver erro de parsing
                    pass

        # ajusta canvas ao tamanho do viewport lido
        vpxmin, vpymin = self.viewport['vpmin']
        vpxmax, vpymax = self.viewport['vpmax']
        largura = max(200, int(abs(vpxmax - vpxmin)))
        altura = max(200, int(abs(vpymax - vpymin)))
        self.canvas.config(width=largura, height=altura)

        # parse window
        w = root.find('window')
        if w is not None:
            wmin = w.find('wmin')
            wmax = w.find('wmax')
            if wmin is not None and wmax is not None:
                try:
                    self.window['wmin'] = (float(wmin.get('x','0')), float(wmin.get('y','0')))
                    self.window['wmax'] = (float(wmax.get('x','1')), float(wmax.get('y','1')))
                except ValueError:
                    pass

        # parse objetos
        self.objects = []
        for child in root:
            tag = child.tag.lower() if child.tag else ''
            # ignorar objetos marcados coords="viewport"
            if child.get('coords') == 'viewport':
                continue
            if tag == 'viewport' or tag == 'window':
                continue
            if tag == 'ponto':
                try:
                    x = float(child.get('x','0'))
                    y = float(child.get('y','0'))
                except ValueError:
                    continue
                self.objects.append({'type':'ponto', 'points':[(x,y)]})
            elif tag == 'reta':
                pts = []
                for p in child.findall('ponto'):
                    try:
                        pts.append((float(p.get('x','0')), float(p.get('y','0'))))
                    except ValueError:
                        continue
                if len(pts) >= 2:
                    self.objects.append({'type':'reta', 'points':pts[:2]})
            elif tag == 'poligono':
                pts = []
                for p in child.findall('ponto'):
                    try:
                        pts.append((float(p.get('x','0')), float(p.get('y','0'))))
                    except ValueError:
                        continue
                if len(pts) >= 3:
                    self.objects.append({'type':'poligono', 'points':pts})

        # calcula os limites do mundo baseado nos objetos carregados
        self.calcular_limites_mundo()

        # monta matriz de transformação world -> viewport
        self.recalcular_matriz()

        # desenha
        self.desenhar_viewport()
        self.desenhar_minimapa()

    # Calcula bounding box dos objetos
    def calcular_limites_mundo(self):
        if not self.objects:
            # usa window como base se não houver objetos
            wxmin, wymin = self.window['wmin']
            wxmax, wymax = self.window['wmax']
            self.world_bounds = {'xmin': wxmin, 'xmax': wxmax, 'ymin': wymin, 'ymax': wymax}
            return
        
        xs = []
        ys = []
        for obj in self.objects:
            for x, y in obj['points']:
                xs.append(x)
                ys.append(y)
        
        if xs and ys:
            xmin, xmax = min(xs), max(xs)
            ymin, ymax = min(ys), max(ys)
            # margem relativa (30% do tamanho atual da window)
            wwidth = self.window['wmax'][0] - self.window['wmin'][0]
            wheight = self.window['wmax'][1] - self.window['wmin'][1]
            margin_x = abs(wwidth) * 0.3 if wwidth != 0 else 1.0
            margin_y = abs(wheight) * 0.3 if wheight != 0 else 1.0
            self.world_bounds = {
                'xmin': xmin - margin_x,
                'xmax': xmax + margin_x,
                'ymin': ymin - margin_y,
                'ymax': ymax + margin_y
            }
            
    # Recalcula a matriz M (world -> viewport).
    def recalcular_matriz(self):
        wxmin, wymin = self.window['wmin']
        wxmax, wymax = self.window['wmax']
        vpxmin, vpymin = self.viewport['vpmin']
        vpxmax, vpymax = self.viewport['vpmax']

        # proteção contra divisão por zero
        if (wxmax - wxmin) == 0:
            sx = 1.0
        else:
            sx = (vpxmax - vpxmin) / (wxmax - wxmin)
        if (wymax - wymin) == 0:
            sy = 1.0
        else:
            sy = (vpymax - vpymin) / (wymax - wymin)

        tx = vpxmin - sx * wxmin
        ty = vpymin + sy * wymax

        self.M = [
            [sx, 0.0, tx],
            [0.0, -sy, ty],
            [0.0, 0.0, 1.0]
        ]
        
    # aplica a transformada de viewport a um ponto e retornar o ponto calculado
    def window2viewport(self, ponto, window=None, viewport=None):
        if window is None:
            window = self.window
        if viewport is None:
            viewport = self.viewport

        xw, yw = ponto
        # aplica matriz M (coordenadas absolutas no sistema do widget)
        xp = self.M[0][0] * xw + self.M[0][1] * yw + self.M[0][2]
        yp = self.M[1][0] * xw + self.M[1][1] * yw + self.M[1][2]
        # converte para coordenadas relativas ao viewport (origem no vpmin)
        vpxmin, vpymin = viewport['vpmin']
        return (xp - vpxmin, yp - vpymin)

    # transforma um ponto do mundo para viewport
    def transformar_ponto_para_viewport(self, x, y):
        return self.window2viewport((x,y))
    
    # Transforma todos os objetos (mundo -> viewport)
    def transformar_objetos_para_viewport(self):
        objetos_transformados = []
        for obj in self.objects:
            obj_transformado = {'type': obj['type'], 'points': []}
            for x, y in obj['points']:
                xp, yp = self.transformar_ponto_para_viewport(x, y)
                obj_transformado['points'].append((xp, yp))
            objetos_transformados.append(obj_transformado)
        return objetos_transformados

    # redensenhar a viewport
    def desenhar_viewport(self):
        self.canvas.delete("all")
        largura = int(self.canvas.cget('width'))
        altura = int(self.canvas.cget('height'))

        # desenha objetos
        for obj in self.objects:
            if obj['type'] == 'ponto':
                xw, yw = obj['points'][0]
                cx, cy = self.window2viewport((xw,yw))
                r = 3
                self.canvas.create_oval(cx-r, cy-r, cx+r, cy+r, fill='black')
            elif obj['type'] == 'reta':
                (x0,y0),(x1,y1) = obj['points'][0], obj['points'][1]
                c0x, c0y = self.window2viewport((x0,y0))
                c1x, c1y = self.window2viewport((x1,y1))
                self.canvas.create_line(c0x, c0y, c1x, c1y, width=2)
            elif obj['type'] == 'poligono':
                pts = []
                for (x,y) in obj['points']:
                    px, py = self.window2viewport((x,y))
                    pts.extend([px, py])
                if len(pts) >= 6:
                    # fecha o polígono
                    self.canvas.create_line(*pts, pts[0], pts[1], width=2)

        # borda da viewport
        self.canvas.create_rectangle(0,0,largura-1, altura-1, outline='black')

        # mostra a matriz no canto superior esquerdo
        txt = (f"Matriz 3x3 (world->viewport):\n"
               f"[{self.M[0][0]:.4f} {self.M[0][1]:.4f} {self.M[0][2]:.4f}]\n"
               f"[{self.M[1][0]:.4f} {self.M[1][1]:.4f} {self.M[1][2]:.4f}]\n"
               f"[{self.M[2][0]:.4f} {self.M[2][1]:.4f} {self.M[2][2]:.4f}]")
        self.canvas.create_text(8, 8, anchor='nw', text=txt, font=("Courier", 9), fill='blue')

    # desenhar a viewport do minimapa
    def desenhar_minimapa(self):
        self.minimap.delete("all")
        xs, ys = [], []
        for obj in self.objects:
            for x,y in obj['points']:
                xs.append(x); ys.append(y)
        # adiciona bounds e window
        xs.extend([self.window['wmin'][0], self.window['wmax'][0],
                   self.world_bounds['xmin'], self.world_bounds['xmax']])
        ys.extend([self.window['wmin'][1], self.window['wmax'][1],
                   self.world_bounds['ymin'], self.world_bounds['ymax']])

        if not xs:
            return

        xmin, xmax = min(xs), max(xs)
        ymin, ymax = min(ys), max(ys)
        if xmin == xmax:
            xmin -= 1; xmax += 1
        if ymin == ymax:
            ymin -= 1; ymax += 1

        # padding
        pad_x = (xmax - xmin) * 0.08
        pad_y = (ymax - ymin) * 0.08
        xmin -= pad_x; xmax += pad_x; ymin -= pad_y; ymax += pad_y

        mw = int(self.minimap.cget('width'))
        mh = int(self.minimap.cget('height'))

        def world_to_minimap(px, py):
            # Adiciona proteção contra divisão por zero
            range_x = xmax - xmin
            range_y = ymax - ymin
            if range_x == 0 or range_y == 0:
                return 0, 0
            sx = mw / range_x
            sy = mh / range_y
            mx = (px - xmin) * sx
            my = mh - (py - ymin) * sy  # inverte y para o minimapa
            return mx, my

        # desenha limites do mundo como retângulo cinza
        p1 = world_to_minimap(self.world_bounds['xmin'], self.world_bounds['ymin'])
        p2 = world_to_minimap(self.world_bounds['xmin'], self.world_bounds['ymax'])
        p3 = world_to_minimap(self.world_bounds['xmax'], self.world_bounds['ymax'])
        p4 = world_to_minimap(self.world_bounds['xmax'], self.world_bounds['ymin'])
        self.minimap.create_polygon(p1[0],p1[1], p2[0],p2[1], p3[0],p3[1], p4[0],p4[1], outline='gray', fill='lightgray', width=1)

        # desenha objetos
        for obj in self.objects:
            if obj['type'] == 'ponto':
                x,y = obj['points'][0]
                mx,my = world_to_minimap(x,y)
                self.minimap.create_oval(mx-2, my-2, mx+2, my+2, fill='black')
            elif obj['type'] == 'reta':
                (x0,y0),(x1,y1) = obj['points'][0], obj['points'][1]
                m0 = world_to_minimap(x0,y0); m1 = world_to_minimap(x1,y1)
                self.minimap.create_line(m0[0], m0[1], m1[0], m1[1], width=1)
            elif obj['type'] == 'poligono':
                pts = []
                for x,y in obj['points']:
                    mx,my = world_to_minimap(x,y)
                    pts.extend([mx,my])
                if len(pts) >= 6:
                    self.minimap.create_line(*pts, pts[0], pts[1], width=1)

        # desenha o retângulo da window atual
        wxmin, wymin = self.window['wmin']
        wxmax, wymax = self.window['wmax']
        p1 = world_to_minimap(wxmin, wymin)
        p2 = world_to_minimap(wxmin, wymax)
        p3 = world_to_minimap(wxmax, wymax)
        p4 = world_to_minimap(wxmax, wymin)
        self.minimap.create_polygon(p1[0],p1[1], p2[0],p2[1], p3[0],p3[1], p4[0], p4[1], outline='red', fill='', width=2)

    # move window e redesenha
    def _mover_e_recarregar(self, dx, dy):
        wminx, wminy = self.window['wmin']
        wmaxx, wmaxy = self.window['wmax']
    
        window_width = wmaxx - wminx
        window_height = wmaxy - wminy

        world_width = self.world_bounds['xmax'] - self.world_bounds['xmin']
        world_height = self.world_bounds['ymax'] - self.world_bounds['ymin']

        # Se a window for maior ou igual ao mundo, ela deve ser centralizada e o movimento não é permitido.
        if window_width >= world_width or window_height >= world_height:
            center_x = (self.world_bounds['xmin'] + self.world_bounds['xmax']) / 2
            center_y = (self.world_bounds['ymin'] + self.world_bounds['ymax']) / 2
        
            # Centraliza a window em relação ao mundo
            new_wminx = center_x - window_width / 2
            new_wminy = center_y - window_height / 2
            new_wmaxx = center_x + window_width / 2
            new_wmaxy = center_y + window_height / 2
        
            self.window['wmin'] = (new_wminx, new_wminy)
            self.window['wmax'] = (new_wmaxx, new_wmaxy)

        # Se a window for menor que o mundo, permite o movimento com limites
        else:
            # calcula o passo do movimento proposto
            movimento = 0.2 
            step_x = window_width * movimento * dx
            step_y = window_height * movimento * dy

            # Garante que o passo do movimento não ultrapasse os limites
            if dx > 0: # Direita
                distancia_max_direita = self.world_bounds['xmax'] - wmaxx
                step_x = min(step_x, distancia_max_direita)
            elif dx < 0: # Esquerda
                distancia_max_esquerda = wminx - self.world_bounds['xmin']
                step_x = max(step_x, -distancia_max_esquerda)

            if dy > 0: # Cima
                distancia_max_cima = self.world_bounds['ymax'] - wmaxy
                step_y = min(step_y, distancia_max_cima)
            elif dy < 0: # Baixo
                distancia_max_baixo = wminy - self.world_bounds['ymin']
                step_y = max(step_y, -distancia_max_baixo)
        
            # Aplica o passo já corrigido
            new_wminx = wminx + step_x
            new_wminy = wminy + step_y
            new_wmaxx = new_wminx + window_width
            new_wmaxy = new_wminy + window_height
        
            self.window['wmin'] = (new_wminx, new_wminy)
            self.window['wmax'] = (new_wmaxx, new_wmaxy)

        # recalcula a matriz e redesenha a tela
        self.recalcular_matriz()
        self.desenhar_viewport()
        self.desenhar_minimapa()

    # função de Zoom redimensiona a window por um fator a partir do seu centro
    def _zoom(self, fator):
        wminx, wminy = self.window['wmin']
        wmaxx, wmaxy = self.window['wmax']

        # Calcula a largura e altura atuais
        width = wmaxx - wminx
        height = wmaxy - wminy

        # Calcula a nova largura e altura propostas
        new_width = width * fator
        new_height = height * fator

        # Limites de tamanho do Zoom
        world_width = self.world_bounds['xmax'] - self.world_bounds['xmin']
        world_height = self.world_bounds['ymax'] - self.world_bounds['ymin']

        if new_width > world_width:
            new_width = world_width
        if new_height > world_height:
            new_height = world_height

        MIN_SIZE = 1e-6
        if new_width < MIN_SIZE or new_height < MIN_SIZE:
            return

        # Calcula o centro da window atual para pivotar o zoom
        center_x = (wminx + wmaxx) / 2
        center_y = (wminy + wmaxy) / 2

        # Calcula as novas coordenadas se estiver fora dos limites
        new_wminx = center_x - new_width / 2
        new_wminy = center_y - new_height / 2
        new_wmaxx = center_x + new_width / 2
        new_wmaxy = center_y + new_height / 2

        # Se a expansão da window ultrapassou os limites do mundo, corrige a posição dela.
        if new_wmaxx > self.world_bounds['xmax']:
            deslocamento = self.world_bounds['xmax'] - new_wmaxx
            new_wmaxx += deslocamento
            new_wminx += deslocamento
        elif new_wminx < self.world_bounds['xmin']:
            deslocamento = self.world_bounds['xmin'] - new_wminx
            new_wminx += deslocamento
            new_wmaxx += deslocamento

        if new_wmaxy > self.world_bounds['ymax']:
            deslocamento = self.world_bounds['ymax'] - new_wmaxy
            new_wmaxy += deslocamento
            new_wminy += deslocamento
        elif new_wminy < self.world_bounds['ymin']:
            deslocamento = self.world_bounds['ymin'] - new_wminy
            new_wminy += deslocamento
            new_wmaxy += deslocamento

        # Atualiza a window com os valores finais e corrigidos
        self.window['wmin'] = (new_wminx, new_wminy)
        self.window['wmax'] = (new_wmaxx, new_wmaxy)

        # Redesenha tudo
        self.recalcular_matriz()
        self.desenhar_viewport()
        self.desenhar_minimapa()

    # função para formatar XML com indentação
    def _indent_xml(self, elem, level=0):
        i = "\n" + level * "\t"
        if len(elem):
            if not elem.text or not elem.text.strip():
                elem.text = i + "\t"
            if not elem.tail or not elem.tail.strip():
                elem.tail = i
            for child in elem:
                self._indent_xml(child, level + 1)
            if not child.tail or not child.tail.strip():
                child.tail = i
        else:
            if level and (not elem.tail or not elem.tail.strip()):
                elem.tail = i

    # gerar XML de saída com dados originais e transformados
    def gerar_xml_saida(self):
        if not self.caminho_arquivo:
            messagebox.showwarning("Aviso", "Nenhum arquivo foi carregado ainda!")
            return
        
        try:
            # solicita onde salvar o arquivo
            caminho_saida = filedialog.asksaveasfilename(
                defaultextension=".xml",
                filetypes=[("XML files", "*.xml"), ("All files", "*.*")],
                title="Salvar XML de Saída"
            )
            if not caminho_saida:
                return
            # usa a árvore do arquivo original para manter metadados, window, viewport, etc.
            tree = ET.parse(self.caminho_arquivo)
            root = tree.getroot()
            
            # obtém objetos transformados
            objetos_transformados = self.transformar_objetos_para_viewport()
            
            # adiciona comentário separador
            comentario = ET.Comment(" Dados transformados para coordenadas da viewport ")
            root.append(comentario)
            # adiciona cada objeto transformado com atributo coords="viewport"
            for obj_trans in objetos_transformados:
                if obj_trans['type'] == 'ponto':
                    x, y = obj_trans['points'][0]
                    ponto_elem = ET.SubElement(root, 'ponto')
                    ponto_elem.set('x', f"{x:.6f}")
                    ponto_elem.set('y', f"{y:.6f}")
                    ponto_elem.set('coords', 'viewport')
                elif obj_trans['type'] == 'reta':
                    reta_elem = ET.SubElement(root, 'reta')
                    reta_elem.set('coords', 'viewport')
                    for x, y in obj_trans['points']:
                        ponto_elem = ET.SubElement(reta_elem, 'ponto')
                        ponto_elem.set('x', f"{x:.6f}")
                        ponto_elem.set('y', f"{y:.6f}")
                        
                elif obj_trans['type'] == 'poligono':
                    poligono_elem = ET.SubElement(root, 'poligono')
                    poligono_elem.set('coords', 'viewport')
                    for x, y in obj_trans['points']:
                        ponto_elem = ET.SubElement(poligono_elem, 'ponto')
                        ponto_elem.set('x', f"{x:.6f}")
                        ponto_elem.set('y', f"{y:.6f}")
            
            # aplica formatação com indentação
            self._indent_xml(root)
            
            # salva o arquivo
            tree.write(caminho_saida, encoding='utf-8', xml_declaration=True)
            messagebox.showinfo("Sucesso", f"XML de saída gerado com sucesso!\nSalvo em: {caminho_saida}")
            
        except Exception as e:
            messagebox.showerror("Erro", f"Erro ao gerar XML de saída:\n{e}")

if __name__ == "__main__":
    root = tk.Tk()
    app = Visualizador(root)
    root.mainloop()